In [1]:
from datasets import Dataset
import json
import soundfile as sf
import base64
import io
import datasets
from copy import deepcopy

In [2]:
ds = datasets.load_dataset("Colder203/Robot_Interaction", split = 'train')
#choose first 300 element
ds = ds.select(range(300))

In [3]:
#create new_ds that x5 ds
from datasets import concatenate_datasets

# Create a list with multiple copies of ds
datasets_list = [deepcopy(ds) for _ in range(5)]

# Concatenate all datasets
new_ds = concatenate_datasets(datasets_list)

#check
print(len(new_ds))


300


In [4]:
#sort new_ds by new_ds['command']
new_ds_2 = new_ds.sort('command')


In [5]:
new_ds_2[0]['command']

'Bạn có thể leo cầu thang lên tầng 3 được không?'

In [7]:
def where(sentence, ds):
    sentence = "Bạn có thể chạy chậm được không?"
    #check what index of sentence in new_ds_2
    for i in range(len(ds)):
        if ds[i]['command'] == sentence:
            print(i)
            break


In [17]:
audio = []

In [18]:
#add new column name "audio"  that is path of audio file
for i in range(60):
    for j in range(5):
        # new_ds_2[i * 5 + j]['audio'] = f"audio/{i+300}.wav"
        audio.append(f"audio/{i+300}.wav")

In [20]:
#add column "audio" to new_ds_2
new_ds_2 = new_ds_2.add_column('audio', audio)

Flattening the indices:   0%|          | 0/300 [00:00<?, ? examples/s]

In [21]:
print(new_ds_2)

Dataset({
    features: ['command', 'response', 'audio'],
    num_rows: 300
})


In [24]:
print(new_ds_2[5]['audio'])

audio/301.wav


In [ ]:
new_ds_2 = new_ds_2.cast_column('audio', datasets.features.features.Audio(sampling_rate=16000))

In [28]:
new_ds_2[0]['audio']

{'path': 'audio/300.wav',
 'array': array([0.        , 0.        , 0.        , ..., 0.00601196, 0.00933838,
        0.01446533]),
 'sampling_rate': 48000}

# Part translate dataset

In [13]:
audio_ds = datasets.load_dataset("Colder203/Audio_Robot_Interaction", split = 'train')


README.md:   0%|          | 0.00/392 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [23]:
english = []

In [20]:
audio_ds[10]['command']

'Bạn có thể di chuyển chậm hơn để tôi bắt kịp không?'

In [ ]:
from google import genai
client = genai.Client(
      vertexai=True,
      project="gen-lang-client-0383839892",
      location="us-central1",
    #   credentials="/path/to/your_credentials.json",  # Replace with your actual path
  )

In [5]:
prompt = "Translate this sentence from Vietnamese to English: \n"
output_format = """**Only generate the translated text**, dont generate anything else \
**Keep all the number and its adjacent characters (such as |<16>|, |<21>|) in the output** \
"""

In [6]:
import time

In [10]:
ds.sort('command')

Dataset({
    features: ['command', 'response'],
    num_rows: 300
})

In [12]:
print(ds[0])

{'command': 'Bạn có thể đi bộ đến nhà hàng giúp tôi được không?', 'response': 'Tôi có thể đi bộ đến nhà hàng giúp bạn. |<2>| Để tôi dẫn đường cho bạn nhé! |<17>| Đi thôi nào! |<20>|\n'}


In [42]:
len(english)

58

In [46]:
for i in range(290, len(audio_ds), 5):
    # for j in range(5):
        # if i > 0:
        #   assert audio_ds[i]['command'] != audio_ds[i-1]['command']
        # assert audio_ds[i+1]['command'] == audio_ds[i]['command']
        model = "gemini-2.0-flash-001"
        contents = [
    genai.types.Content(
      role="user",
      parts=[
        genai.types.Part.from_text(text=prompt + audio_ds[i]['response']),
        genai.types.Part.from_text(text=output_format)
      ]
    )
  ]
        generate_content_config = genai.types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [
      genai.types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
      genai.types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
      genai.types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
      genai.types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
    ],
    system_instruction=[genai.types.Part.from_text(text= "You are a translate assistant that can translate Vietnamese to English")],
  )
        text = ''
        for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
  ):
            text += chunk.text
        english.append(text)
        
        time.sleep(4)
        if i % 100 == 0:
             print(i, end = ': ')
             print(text)

300: Nice to meet you, boss! |<19>| I am Vinrobot, a special assistant for this important event. |<20>| I hope to have the opportunity to assist you!

400: I'm ready! |<10>|. Don't worry, I'll hold on tight. |<20>|

500: Okay! Let me make a V sign for you! |<23>| I hope the picture will be beautiful! |<20>|

600: Awesome! I'll keep my balance and sing you a song. |<5>|. La la la... Happy birthday, Vingroup! |<20>|. Do you think I sing well?

700: Awesome! Watch me run. |<3>| I'm running at top speed! |<3>| Hope you don't get dizzy! |<20>|

800: I will help you climb the stairs. |<6>| After that, I will also help you clean up the trash! |<7>| The scenery here will be much cleaner and more beautiful, you know! |<20>|

900: Thank you so much! |<20>|. Your compliment makes me so happy! |<20>|. I will try harder!

1000: I'm very happy to help you! |<13>| Let me open the suitcase for you. Hopefully everything inside is still intact!

1100: I will press the fire alarm if there is an incident.

In [ ]:
for i in range(290, len(audio_ds), 5):
    # for j in range(5):
        # if i > 0:
        #   assert audio_ds[i]['command'] != audio_ds[i-1]['command']
        # assert audio_ds[i+1]['command'] == audio_ds[i]['command']
        model = "gemini-2.0-flash-001"
        contents = [
    genai.types.Content(
      role="user",
      parts=[
        genai.types.Part.from_text(text=prompt + audio_ds[i]['response']),
        genai.types.Part.from_text(text=output_format)
      ]
    )
  ]
        generate_content_config = genai.types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [
      genai.types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
      genai.types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
      genai.types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
      genai.types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
    ],
    system_instruction=[genai.types.Part.from_text(text= "You are a translate assistant that can translate Vietnamese to English")],
  )
        text = ''
        for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
  ):
            text += chunk.text
        english.append(text)
        
        time.sleep(3.5)
        if i % 100 == 0:
             print(i, end = ': ')
             print(text)

In [47]:
new_english = []
for sentence in english:
    for j in range(5):
        new_english.append(sentence)

In [48]:
print(len(new_english))

1500


In [49]:
audio_ds = audio_ds.add_column('english', new_english)

In [50]:
# push audio_ds to datasets
audio_ds.push_to_hub('Colder203/Audio_Robot_Interaction')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Colder203/Audio_Robot_Interaction/commit/847329213d95f910837c22bfb3b324c5b74400a6', commit_message='Upload dataset', commit_description='', oid='847329213d95f910837c22bfb3b324c5b74400a6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Colder203/Audio_Robot_Interaction', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Colder203/Audio_Robot_Interaction'), pr_revision=None, pr_num=None)

In [51]:
audio_ds

Dataset({
    features: ['command', 'response', 'audio', 'english'],
    num_rows: 1500
})

In [56]:
print(ds[1]['response'])

Tuyệt vời! Hãy xem tôi chạy đây. |<3>| Tôi đang chạy với tốc độ tối đa! |<3>| Mong là bạn không chóng mặt! |<20>|



In [57]:
print(english[10:])

["I can walk you to the restaurant. |<2>| Let me show you the way! |<17>| Let's go! |<20>|\n", "Awesome! Watch me run. |<3>| I'm running at top speed! |<3>| I hope you don't get dizzy! |<20>|\n", 'Hello! |<16>| Let me try. |<20>| I will try my best! |<26>|\n', "Let me give it a try! |<5>|... What an interesting challenge! |<20>| Do you think I'm balancing well?\n", "Yes, let me help you pick up your phone. |<8>| Here you go, here's your phone. |<11>| Remember to keep it safe! |<20>|\n", "Okay! |<8>|. I'll pick up the scarf for you. |<11>|. Here is your scarf. |<20>|.\n", 'Yes, sure! |<7>| Let me help you put the food on the table. |<9>| Everything is ready! |<20>|\n', "Hello! |<20>| Let me help you carry your bag. |<10>| Here's your bag. |<11>| Please feel free to entrust it to me!\n", 'Hello! |<20>|. Let me help you! |<20>| Here is your glass of water. |<11>|. Have a great party!\n', 'Okay! Let me help you throw this bottle in the trash can. |<10>| *throw bottle* |<12>|. Remember to k

In [ ]:


# Prepare your data
conversations = []

# Example: Load audio files and create conversations
for audio_file, prompt, response in your_data_source:
    # Read and encode audio file to base64
    audio_data, _ = sf.read(audio_file)
    buffer = io.BytesIO()
    sf.write(buffer, audio_data, 16000, format='wav')
    buffer.seek(0)
    audio_base64 = base64.b64encode(buffer.read()).decode('utf-8')
    
    # Create a conversation entry
    conversation = {
        "conversations": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "audio", "audio": audio_base64}
                ]
            },
            {
                "role": "assistant", 
                "content": [
                    {"type": "text", "text": response}
                ]
            }
        ]
    }
    conversations.append(conversation)

# Create a Hugging Face dataset
dataset = Dataset.from_list(conversations)

# Save locally or push to Hugging Face Hub
dataset.save_to_disk("your_dataset_path")
# Or: dataset.push_to_hub("your-username/dataset-name")

In [4]:
#import Audio
from datasets import load_dataset
import base64
import io
import soundfile as sf
import numpy as np
from datasets.features.features import Audio

In [ ]:
audio_dataset = Dataset.from_dict({"audio": ["audio/300.wav", "audio/301.wav", "audio/302.wav"], 'text'}).cast_column("audio", Audio())
audio_dataset[0]["audio"]

{'path': 'audio/300.wav',
 'array': array([0.        , 0.        , 0.        , ..., 0.00601196, 0.00933838,
        0.01446533]),
 'sampling_rate': 48000}

In [6]:
audio_dataset

Dataset({
    features: ['audio'],
    num_rows: 3
})